In [1]:
# Packages
import pandas as pd

from bs4 import BeautifulSoup as bs
import requests

In [11]:
def fundamental_metric(soup, metric):
    return soup.find(text = metric).find_next(class_='snapshot-td2').text

def get_fundamental_data(df):
    for symbol in df.index:
        try:
            url = ("http://finviz.com/quote.ashx?t=" + symbol.lower())
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}
            soup = bs(requests.get(url, headers=headers).content)
            for m in df.columns:
                df.loc[symbol, m ] = fundamental_metric(soup, m)
        except Exception as e:
            print(symbol, 'not found')
    return df

In [31]:
stock_list = ['AMZN','GOOG','PG','KO','IBM','DG','XOM','KO','PEP','MT','NL','RTX','LPL']

metric = ['Price',
'P/B',
'P/E',
'Forward P/E',
'PEG',
'Debt/Eq',
'EPS (ttm)',
'Dividend %',
'ROE',
'ROI',
'EPS Q/Q',
'Insider Own'
]

In [37]:
df = pd.DataFrame(index = stock_list, columns = metric)
df = get_fundamental_data(df)

for char in ["%", " ", "/"]:
    df.columns = df.columns.str.replace(char, "")

for col in ['Dividend', 'ROE', 'ROI', 'EPSQQ', 'InsiderOwn']:
    df[col] = df[col].str.replace("%", '')
df = df.apply(pd.to_numeric, errors='coerce')

In [35]:
df

,Price,PB,PE,ForwardPE,PEG,DebtEq,EPS(ttm),Dividend,ROE,ROI,EPSQQ,InsiderOwn
AMZN,3014.82,20.87,118.32,69.70,3.28,0.69,26.02,NaN,20.5,11.7,97.4,10.90
GOOG,1505.69,5.00,33.43,26.95,5.49,NaN,45.49,NaN,NaN,NaN,-30.0,5.75
PG,137.56,7.49,27.77,23.90,3.60,0.76,4.96,2.30,28.3,16.1,153.6,0.10
KO,50.51,12.48,23.89,24.57,8.13,2.99,2.13,3.23,50.1,12.0,-31.9,0.30
IBM,125.14,5.38,13.85,10.17,5.39,3.15,8.97,5.25,39.7,11.4,-46.1,0.10
DG,203.68,6.90,22.03,20.79,1.49,0.00,9.21,0.71,33.5,18.9,89.7,0.20
XOM,37.57,0.90,25.11,27.90,1.94,0.39,1.51,9.20,3.9,2.3,-134.5,0.10
KO,50.51,12.48,23.89,24.57,8.13,2.99,2.13,3.23,50.1,12.0,-31.9,0.30
PEP,134.24,14.98,27.49,22.72,5.02,3.60,4.91,3.03,49.9,17.8,-18.2,0.14
MT,13.43,0.41,NaN,11.22,NaN,0.38,-4.00,1.54,-11.0,-2.1,-13.3,44.80


In [28]:

df_filtered = df[(df['PE'].astype(float)<20) & (df['PB'].astype(float) < 1)]
df_filtered

,PB,PE,ForwardPE,PEG,DebtEq,EPS(ttm),Dividend,ROE,ROI,EPSQQ,InsiderOwn
NL,0.65,17.4,6.18,NaN,0.0,0.23,3.98,3.7,-0.7,-22.0,0.2


In [42]:

df_filtered = df[(df['InsiderOwn'].astype(float)>30) & (df['PB'].astype(float) < 3)]
df_filtered

,Price,PB,PE,ForwardPE,PEG,DebtEq,EPS(ttm),Dividend,ROE,ROI,EPSQQ,InsiderOwn
MT,13.55,0.41,NaN,11.22,NaN,0.38,-4.00,1.54,-11.0,-2.1,-13.3,44.8
LPL,6.92,0.54,NaN,19.19,NaN,1.36,-3.42,NaN,-24.9,-10.8,10.8,51.1
